# deepcopy, lightGBM and pickle
> At first sight, these 3 things may not sounds related at all. I am writing this article to share a bug with lightgbm that I encountered and it eventually leads to deeper understanding of what `pickle` really are.


- toc: true 
- badges: true
- comments: true
- author: noklam
- categories: ["python", "pickle", "deepcopy"]


To start with, let's look at some code to get some context.

In [6]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from copy import deepcopy

params = {
'objective': 'regression',
'verbose': -1,
'num_leaves': 3
}

X = np.random.rand(100,2)
Y = np.ravel(np.random.rand(100,1))
lgbm = lgb.train(params, lgb.Dataset(X,label=Y),num_boost_round=1)
print("Parameters of the model: ", lgbm.params)

Parameters of the model:  {'objective': 'regression', 'verbose': -1, 'num_leaves': 3, 'num_iterations': 1, 'early_stopping_round': None}


In [7]:
## Deep copy will missing params
new_model = deepcopy(lgbm)

Finished loading model, total used 1 iterations


You would expect `new_model.parameters` return the same dict right? Not quite.

In [8]:
print("Parameters of the copied model: ", new_model.params)

Parameters of the copied model:  {}


Surprise, surprise. It's an empty dict, where did the parameters go? To dive deep into the issue, let's have a look at the source code of `deepcopy` to understand how does it work.

reference: https://github.com/python/cpython/blob/e8e341993e3f80a3c456fb8e0219530c93c13151/Lib/copy.py#L128

```python
def deepcopy(x, memo=None, _nil=[]):
    """Deep copy operation on arbitrary Python objects.
    See the module's __doc__ string for more info.
    """

    ... # skip some irrelevant code  

    cls = type(x)

    copier = _deepcopy_dispatch.get(cls)
    if copier is not None:
        y = copier(x, memo)
    else:
        if issubclass(cls, type):
            y = _deepcopy_atomic(x, memo)
        else:
            copier = getattr(x, "__deepcopy__", None)
            if copier is not None:
                y = copier(memo)
            else:
                ... # skip irrelevant code

    # If is its own copy, don't memoize.
    if y is not x:
        memo[d] = y
        _keep_alive(x, memo) # Make sure x lives at least as long as d
    return y
```

In particular, line 17 is what we care.  
`copier = getattr(x, "__deepcopy__", None)`  


If a particular class has implement the `__deepcopy__` method, `deepcopy` will try to invoke that instead of the standard copy. The following dummy class should illustrate this clearly.

In [15]:
class DummyClass():
    def __deepcopy__(self, _):
        print('Just hanging around and not copying.')

In [16]:
o = DummyClass()
deepcopy(o)

Just hanging around and not copying.


a lightgbm model is actually a `Booster` object and implement its own `__deepcopy__`. It only copy the model string but nothing else, this explains why `deepcopy(lgbm).paramters` is an empty dictionary.

```python
 def __deepcopy__(self, _): 
     model_str = self.model_to_string(num_iteration=-1) 
     booster = Booster(model_str=model_str) 
     return booster 
```


Reference: https://github.com/microsoft/LightGBM/blob/d6ebd063fff7ff9ed557c3f2bcacc8f9456583e6/python-package/lightgbm/basic.py#L2279-L2282

Okay, so why `lightgbm` need to have an custom implementation? I thought this is a bug, but turns out there are some deeper reason behind this.
I created an issue on GitHub.

https://github.com/microsoft/LightGBM/issues/4085
Their response is 
> Custom __deepcopy__ is needed to make `Booster` class picklable.

# 🥖Italian BMT, 🥬Lettuce  🍅 tomato and some 🥒pickles please

What does `pickle` really is? and what makes an object `pickable`?

> Python Pickle is used to serialize and deserialize a python object structure. Any object on python can be pickled so that it can be saved on disk.

Serialization roughly means translating the data in memory into a format that can be stored on disk or sent over network. It's like ordering a chair from Ikea, they will send you a box, but not a chair.

The process of decomposing the chair and put it into a box is `serialization`, while putting it together is `deserialization`.

![deserialize and serialize](images/serialize.jpg "Putting the chair together is `deseralization`, decomposing the chair and make it packable in a box is `serialization`")